In [22]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

input_df = pd.read_csv(r'../datasets/input_df.csv')
# print(input_df)


In [25]:
print(input_df)

     Rutting (mm)  Fatigue_Cracking (m²)  Block_Cracking (m²)  \
4.8           2.0                    0.0                 26.0   

     Longitudinal_Cracking (m²)  Transverse_Cracking (m²)  Patching (m²)  \
4.8                         0.0                     118.0              5   

     Potholes (Number)  Delamination (m²)  Severity_Rating  \
4.8               18.0               20.0                0   

     Traffic_Volume (vehicles/day)  Temperature_C  Precipitation_mm  \
4.8                           8600           10.5             120.0   

     Maintenance_History  
4.8                    0  


In [24]:

label_encoder = LabelEncoder()
label_encoder.fit(["Low", "Medium", "High"])
input_df['Severity_Rating'] = label_encoder.fit_transform(input_df['Severity_Rating'])
label_encoder2 = LabelEncoder()
label_encoder2.fit(["None", "Minor repairs", "Major repairs"])
input_df['Maintenance_History'] = label_encoder2.fit_transform(input_df['Maintenance_History'])


In [38]:
scaler = MinMaxScaler()
numeric_columns = ['Rutting (mm)', 'Fatigue_Cracking (m²)', 'Block_Cracking (m²)', 
                   'Longitudinal_Cracking (m²)', 'Transverse_Cracking (m²)', 
                   'Patching (m²)', 'Potholes (Number)', 'Delamination (m²)', 
                   'Traffic_Volume (vehicles/day)', 'Temperature_C', 'Precipitation_mm','Severity_Rating','Maintenance_History']

# input_df[numeric_columns] = scaler.fit_transform(input_df[numeric_columns])

# print((input_df[numeric_columns]))
# for i in input_df:
#     print(input_df[i])
# print(input_df)
print(scaler.fit_transform(input_df[numeric_columns]))
# print(scaler.min_)
# print(scaler)
# input_df[numeric_columns] = scaler.transform(input_df[numeric_columns])
# print(scaler.transform(input_df[numeric_columns]))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
